# Text Feature Engineering

In [19]:
import re
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

In [21]:
# nltk.download('wordnet')

### Split the data in a training set to apply transformations

In [2]:
df = pd.read_csv('csv_files/9_1_all_books_df.csv')

In [3]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['description'], 
                                                                    df['best_seller'], 
                                                                    test_size = .2,
                                                                    random_state = 42)

In [4]:
# T_train on it's own is a series
type(X_train)

pandas.core.series.Series

# one hot encoding categories
- genres
- format
- publisher
- month (most likely not)

# vectorize the description
- look at html format
- key words
- take out NYT reference

### cleaning html tags
- as well as exploring vectorizing in pandas.

In [6]:
# no_html = df.description[3].replace('<i>', '')
# no_html
df.description[1371]
# this is what we want to act on
# df.description.values

# use regrex to clean the html tags
# no_reg_html = re.sub('<.{1,9}>',' ', df.description[1372])
# no_reg_html
# vec_html = re.sub('<.{1,9}>',' ', df.description.values)

'<b>Nonpareil science writer David Quammen explains how recent discoveries in molecular biology can change our understanding of evolution and life’s history, with powerful implications for human health and even our own human nature. </b>'

#### function to take out the tags

In [7]:
def no_html(text):
    return re.sub('<.{1,9}>',' ', text)

#### function to remove years and numbers in general

In [8]:
def no_nums(text):
    return re.sub('\d+', ' ', text)

#### method 1 np.vectorize

In [9]:
no_html_v = np.vectorize(no_html)

In [10]:
no_html_v([X_train[1371],df.description[1372] ])

array([' Nonpareil science writer David Quammen explains how recent discoveries in molecular biology can change our understanding of evolution and life’s history, with powerful implications for human health and even our own human nature.  ',
       ' NAMED A BEST BOOK OF 2018 BY  THE NEW YORK TIMES  "Somehow Casey Gerald has pulled off the most urgently political, most deeply personal, and most engagingly spiritual statement of our time by just looking outside his window and inside himself. Extraordinary." - Marlon James "Staccato prose and peripatetic storytelling combine the cadences of the Bible with an urgency reminisc '],
      dtype='<U381')

In [11]:
X_train = no_html_v(X_train)

In [ ]:
# some series magic... originally made the function for a dataframe
# X_train["no_html"] = no_html_v(X_train.values)

In [74]:
# X_train = X_train['no_html']

In [13]:
X_train

array(['An exquisite memoir about how to live--and love--every day with "death in the room," from poet Nina Riggs, mother of two young sons and the direct descendant of Ralph Waldo Emerson, in the tradition of ',
       "The Vision wants to be human, and what's more human than family? So he heads back to the beginning, to the laboratory where Ultron created him and molded him into a weapon. The place where he first rebelled against his given destiny and imagined that he could be more -that he could be a man. There, he builds them. A wife, Virginia. Two teenage twins, Viv and Vin. They look",
       ' "An intelligent explanation of the mechanisms that produced the crisis and the response to it...One of the great strengths of Tooze\'s book is to demonstrate the deeply intertwined nature of the European and American financial systems."   --The New York Times Book Review       From a prizewinning economic historian, an eye-opening reinterpretation of the 2008 economic crisis ( ',
       ..

#### apply numpy function

In [14]:
no_nums_v = np.vectorize(no_nums)

In [15]:
X_train = no_nums_v(X_train)

#### method 2 pandas apply, this doesn't not apply to X_train

In [16]:
# df['no_html_apply'] = df.description.apply(no_html)

# df.head()

## Cleaning text
- punctuation
- lowercase
- lemmas

In [105]:
cv = CountVectorizer(stop_words='english', tokenizer=Nyt_LemmaTokenizer())

In [161]:
# v_X_train = cv.fit_transform(X_train)

In [84]:
# act on the CountVectorizer object to get feature names
# this is the first past, with numbers and no stop words
len(cv.get_feature_names())
# number of words 7162

7162

In [87]:
# second pass with stop words
len(cv.get_feature_names())
# number of words 6902

6902

In [103]:
# third pass, no numbers and stop words
print(len(cv.get_feature_names()))
# cv.get_feature_names()
# number of words 6799

6799


### POS for lemmas

In [109]:
nltk.pos_tag(['5'])[0][1][0]
# ADJ = adjective
# NOUN = noun
# VERB = verb
# ADV = adverb

'C'

In [30]:
lemmatizer = WordNetLemmatizer()
# n = noun
# v = verb
# a = adjective
# r = adverb

In [102]:
def mapping_pos(word):
    if word.isdigit():
        return ' '
    else:
        tag = nltk.pos_tag([word])[0][1][0]
        if tag == 'J':
            return 'a'
        elif tag == 'V':
            return 'v'
        elif tag == 'R':
            return 'r'
        else:
            return 'n'

In [103]:
mapping_pos('5')

' '

In [79]:
lemmatizer.lemmatize('bigger', mapping_pos('bigger'))
lemmatizer.lemmatize('twin', mapping_pos('twin'))

'twin'

In [129]:
kids = ['girls', 'boys']
size = ['bigger', 'biggest']
def lemmtize_it (sentences):
    # split the string of sentences
    sentence = sentences.split(' ')
    cleaned = [lemmatizer.lemmatize(i, mapping_pos(i)) for i in sentence]
    return " ".join(cleaned)



In [104]:
class Nyt_LemmaTokenizer(object):
    def __call__(self, sentences):
        sentence = sentences.split(' ')
        return [lemmatizer.lemmatize(i, mapping_pos(i)) for i in sentence]

In [155]:
# lemmtize_it(X_train[2])

In [127]:
X_train[0]

'An exquisite memoir about how to live--and love--every day with "death in the room," from poet Nina Riggs, mother of two young sons and the direct descendant of Ralph Waldo Emerson, in the tradition of '

In [123]:
v_lemmtize_it =  np.vectorize(lemmtize_it)

In [124]:
v_lemmtize_it(X_train)

IndexError: string index out of range

In [108]:
# fourth pass will use lemmas


## Stop words

In [6]:
# this base of stop words is a frozen set
text.ENGLISH_STOP_WORDS


frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

# testing out a sentence to vectorize

In [128]:
X_train[0:5]

array(['An exquisite memoir about how to live--and love--every day with "death in the room," from poet Nina Riggs, mother of two young sons and the direct descendant of Ralph Waldo Emerson, in the tradition of ',
       "The Vision wants to be human, and what's more human than family? So he heads back to the beginning, to the laboratory where Ultron created him and molded him into a weapon. The place where he first rebelled against his given destiny and imagined that he could be more -that he could be a man. There, he builds them. A wife, Virginia. Two teenage twins, Viv and Vin. They look",
       ' "An intelligent explanation of the mechanisms that produced the crisis and the response to it...One of the great strengths of Tooze\'s book is to demonstrate the deeply intertwined nature of the European and American financial systems."   --The New York Times Book Review       From a prizewinning economic historian, an eye-opening reinterpretation of the   economic crisis ( ',
       ' In 

In [153]:
type(X_train[2])

numpy.str_

In [154]:
X_train[2]

' "An intelligent explanation of the mechanisms that produced the crisis and the response to it...One of the great strengths of Tooze\'s book is to demonstrate the deeply intertwined nature of the European and American financial systems."   --The New York Times Book Review       From a prizewinning economic historian, an eye-opening reinterpretation of the   economic crisis ( '

In [158]:
# this doesn't work... need to process charaters before
lemmtize_it(X_train[1])

"The Vision want to be human, and what's more human than family? So he head back to the beginning, to the laboratory where Ultron create him and mold him into a weapon. The place where he first rebel against his give destiny and imagine that he could be more -that he could be a man. There, he build them. A wife, Virginia. Two teenage twins, Viv and Vin. They look"

In [163]:
re.sub
# X_train[0].replace('.')

['An exquisite memoir about how to live--and love--every day with "death in the room," from poet Nina Riggs, mother of two young sons and the direct descendant of Ralph Waldo Emerson, in the tradition of ']

In [151]:
lemmatizer.lemmatize('going')

'going'